# Biomass Field Data Clean v4

Extracts species level basal data from an excel spreadsheet and outputs a data frame called slats_c_bio_site_totals_v(n).csv

The notebook calculates site proportions, STBA, LTBA and DTBA, applies the allometric cofactors from Cook et al. 2005) to calculate Carbon extent then applies the secondary cofactor to convert C extent to AGB extent.

The following conditions apply:

 - env = base
 - run on site basal count data (e.g. slats_biomass_field_data.xlsx)

In [2]:
import pandas as pd
import geopandas as gpd
import os

In [3]:
date = "20221221"

In [4]:
dir_ = r"F:\cdu\data\slats\varified_by_rm"
output_dir = r"F:\cdu\data\output\{0}".format(date)

In [5]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [6]:
os.listdir(dir_)

['slats_biomass_field_data_script.xlsx', 'slats_biomass_field_data.xlsx']

In [7]:
excel_ = os.path.join(dir_, "slats_biomass_field_data.xlsx")

In [8]:
df = pd.read_excel(excel_, sheet_name=0, index_col =0)
df

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,site,date_er,date,zone,easting,northing,lattitude,longitude,datum,factor,...,nw_d_sp08,nw_sp09,nw_l_sp09,nw_d_sp09,nw_sp10,nw_l_sp10,nw_d_sp10,nw_sp11,nw_l_sp11,nw_d_sp11
uid,,,,,,,,,,,,,,,,,,,,,
0,lit01,26/04/2013,2013-04-26,52.0,724039.000,8611982.000,NaN,NaN,GDA94,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agb02,17/04/2012,2012-04-17,52.0,717789.008,8601033.837,NaN,NaN,WGS84,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,buff01,13/07/2012,2021-07-13,52.0,706454.000,8635540.000,NaN,NaN,WGS84,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ep01,13/07/2012,2012-07-13,52.0,698028.000,8627310.000,NaN,NaN,WGS84,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,girra01,2012-05-06 00:00:00,2012-06-05,52.0,725819.000,8615707.000,NaN,NaN,WGS84,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,auv09,2012-10-11 00:00:00,2012-10-11,52.0,569278.000,8212966.000,NaN,NaN,GDA94,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,nt001,2011-05-23 00:00:00,2011-05-23,52.0,738723.000,8457225.000,NaN,NaN,GDA94,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,nt002,2012-05-23 00:00:00,2012-05-23,52.0,739680.000,8458420.000,NaN,NaN,GDA94,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# original filed data test (13 sites)
#csv_ = os.path.join(dir_, "tree_biomass_field_data_copy.csv")
# merged csv abova and obs sheet 2013
# csv_ = os.path.join(dir_, "tree_biomass_field_data_20220917.csv")

In [10]:
#csv_ = "\\pgb-bas01\DENR_Satellite_Imagery$\Scratch\Rob\tern\tree_biomass_field_data\biomass_carbon\tree_biomass_field_data_copy.csv"

In [11]:
#df = pd.read_excel(csv_, sheet_name="tree_biomass_field_data", encoding='windows-1252')
# df = pd.read_csv(csv_, encoding='windows-1252',delimiter='\s+')

In [12]:
df.shape

(122, 246)

In [13]:
print(len(df.site.unique()))

122


In [14]:
def am_date(df):
    
    am_date_list = []
    if not "am_date" in df.columns:
        for i in df.date:
            print(i)
            d = i.strftime("%Y%m%d")
            am_date_list.append(d)
            print(d)
    df.insert(3, "am_date", am_date_list)        
    return df
        

In [15]:
df = am_date(df)

2013-04-26 00:00:00
20130426
2012-04-17 00:00:00
20120417
2021-07-13 00:00:00
20210713
2012-07-13 00:00:00
20120713
2012-06-05 00:00:00
20120605
2012-06-05 00:00:00
20120605
2012-06-06 00:00:00
20120606
2012-07-10 00:00:00
20120710
2012-07-10 00:00:00
20120710
2012-06-06 00:00:00
20120606
2012-07-10 00:00:00
20120710
2013-06-02 00:00:00
20130602
2012-06-02 00:00:00
20120602
2013-07-16 00:00:00
20130716
2013-07-16 00:00:00
20130716
2013-07-16 00:00:00
20130716
2013-05-31 00:00:00
20130531
2013-06-01 00:00:00
20130601
2013-06-01 00:00:00
20130601
2013-05-28 00:00:00
20130528
2013-05-28 00:00:00
20130528
2013-05-24 00:00:00
20130524
2013-05-24 00:00:00
20130524
2013-06-03 00:00:00
20130603
2012-06-19 00:00:00
20120619
2012-06-19 00:00:00
20120619
2012-06-20 00:00:00
20120620
2012-06-21 00:00:00
20120621
2012-06-21 00:00:00
20120621
2012-06-22 00:00:00
20120622
2012-06-22 00:00:00
20120622
2012-06-22 00:00:00
20120622
2012-06-26 00:00:00
20120626
2012-06-26 00:00:00
20120626
2012-06-26 00:

In [16]:
def init_clean(df):
    
    """ This function cleans the file by dropping null easting values, 
    filling in the uid column based on the index and fills any missing datum value with 
    WGS84 (data with missing values was collected in 2013)."""
    
    # fill in uid column
    df['uid'] = df.index + 1
    # drop any value where an easting is missing (i.e. no coordinates)
    df.dropna(axis=0, subset=['easting'], inplace= True)
    
    # fill in datum to WGS84 - due to data collection in 2012 2013
    df['datum'] = df['datum'].fillna('WGS84')
    
    return df
     

In [17]:
clean_df = init_clean(df)

In [18]:
clean_df.shape

(120, 248)

In [19]:
def filter_dataframe(df):
    
    """ This function filters the dataframe based on datum and zone information returning four df. """
    
    wgs = df[df['datum']== 'WGS84']
    print("wgs: ", wgs.shape)
    gda = df[df['datum']== 'GDA94']
    print("gda: ", gda.shape)

    wgs52 = wgs[wgs['zone'] == 52.0]
    print("wgsz52: ", wgs52.shape)
    wgs53 = wgs[wgs['zone'] == 53.0]
    print("wgsz53: ", wgs53.shape)
    gda52 = gda[gda['zone'] == 52.0]
    gda53 = gda[gda['zone'] == 53.0]
    
    return [gda52, gda53, wgs52, wgs53]

In [20]:
df_list = filter_dataframe(clean_df)

wgs:  (67, 248)
gda:  (53, 248)
wgsz52:  (56, 248)
wgsz53:  (11, 248)


In [21]:
def convert_gdf(df_list, epsg_list):
    
    """ This function Converts each dataframe to geographics GDA94 and returns a list of active df. """
    
    gdf_list = []
    for df, epsg in zip(df_list, epsg_list):
        gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.easting, df.northing))

        gdf = gdf.set_crs(epsg= epsg) #'epsg:{0}'.format(str(epsg)))

        gdf = gdf.to_crs(4283)

        print(gdf.crs)
        gdf_list.append(gdf)
          
    return gdf_list

In [22]:
gdf_list = convert_gdf(df_list, [28352, 28353, 32752, 32753])

epsg:4283
epsg:4283
epsg:4283
epsg:4283


In [23]:
for i in gdf_list:
    print(i.shape)

(21, 249)
(32, 249)
(56, 249)
(11, 249)


In [24]:
def concat_and_clean(gdf_list):
    """ This function creates a latitude and longitude column and 
    insert the relevant gda94 coordinates and removes other existing coordinate columns. """
    
    final = pd.concat(gdf_list)
    
    final['lon_gda94'] = final.geometry.x
    final['lat_gda94'] = final.geometry.y

    del final['zone']
    del final['easting']
    del final['northing']
    del final['lattitude']
    del final['longitude']
    
    final['datum'] = 'GDA94'
    
    return final
    

In [25]:
clean_gdf = concat_and_clean(gdf_list)
print(clean_gdf.shape)
print(len(clean_gdf.site.unique()))

(120, 246)
120


C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\geopandas\geodataframe.py:853: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)


##### Export to csv

In [26]:
clean_df_ = pd.DataFrame(clean_gdf)
clean_df = clean_df_.reset_index(drop=True)
# fill in uid column
clean_df['uid'] = clean_df.index + 1
clean_df

,site,date_er,date,am_date,datum,factor,count,height_o,height_g,Discription,...,nw_sp10,nw_l_sp10,nw_d_sp10,nw_sp11,nw_l_sp11,nw_d_sp11,uid,geometry,lon_gda94,lat_gda94
0,lit01,26/04/2013,2013-04-26,20130426,GDA94,0.50,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,POINT (131.06192 -12.54791),131.061917,-12.547912
1,girra02,2012-05-06 00:00:00,2012-06-05,20120605,GDA94,0.10,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (131.13200 -12.52341),131.132000,-12.523407
2,umb07,2013-03-06 00:00:00,2013-06-03,20130603,GDA94,0.25,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,POINT (131.67450 -13.96230),131.674500,-13.962300
3,jdr01,2012-05-22 00:00:00,2012-05-22,20120522,GDA94,0.75,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,POINT (131.59970 -13.88379),131.599700,-13.883786
4,jdr02,2012-05-23 00:00:00,2012-05-23,20120523,GDA94,0.50,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,POINT (131.60025 -13.85727),131.600246,-13.857266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,ant09,2012-06-26 00:00:00,2012-06-26,20120626,GDA94,0.00,7,0.0,0.3,pasture - grasslands,...,NaN,NaN,NaN,NaN,NaN,NaN,116,POINT (135.26981 -18.32964),135.269807,-18.329637
116,ant07,2012-06-26 00:00:00,2012-06-26,20120626,GDA94,0.00,7,0.0,0.2,pasture - grasslands,...,NaN,NaN,NaN,NaN,NaN,NaN,117,POINT (135.20384 -18.24580),135.203840,-18.245798
117,ant06,2012-06-25 00:00:00,2012-06-25,20120625,GDA94,0.25,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,118,POINT (135.15252 -18.26631),135.152522,-18.266308
118,barkely02,2011-10-26 00:00:00,2011-10-26,20111026,GDA94,0.00,7,0.0,0.3,pasture - grasslands - no trees,...,NaN,NaN,NaN,NaN,NaN,NaN,119,POINT (135.25977 -18.05696),135.259768,-18.056963


In [27]:
clean_df = pd.DataFrame(clean_gdf)
#slats
clean_df.to_csv(os.path.join(output_dir, "initial_biomass_cleaned_v6.csv"), index=False)
#rm year
#clean_df.to_csv(os.path.join(output_dir, "rm_2014_initial_biomass_cleaned_v1.csv"), index=False)

In [28]:
def search_for_names(gdf, var_):
    """ This function returns a list of column names that includes the substring (var_) """
    var_cols = [col for col in gdf.columns if var_ in col]

    return var_cols

In [29]:
var_cols =search_for_names(clean_gdf, "sp")

In [30]:
print(var_cols)

['cent_sp01', 'cent_l_sp01', 'cent_d_sp01', 'cent_sp02', 'cent_l_sp02', 'cent_d_sp02', 'cent_sp03', 'cent_l_sp03', 'cent_d_sp03', 'cent_sp04', 'cent_l_sp04', 'cent_d_sp04', 'cent_sp05', 'cent_l_sp05', 'cent_d_sp05', 'cent_sp06', 'cent_l_sp06', 'cent_d_sp06', 'cent_sp07', 'cent_l_sp07', 'cent_d_sp07', 'cent_sp08', 'cent_l_sp08', 'cent_d_sp08', 'cent_sp09', 'cent_l_sp09', 'cent_d_sp09', 'cent_sp10', 'cent_l_sp10', 'cent_d_sp10', 'cent_sp11', 'cent_l_sp11', 'cent_d_sp11', 'north_sp01', 'north_l_sp01', 'north_d_sp01', 'north_sp02', 'north_l_sp02', 'north_d_sp02', 'north_sp03', 'north_l_sp03', 'north_d_sp03', 'north_sp04', 'north_l_sp04', 'north_d_sp04', 'north_sp05', 'north_l_sp05', 'north_d_sp05', 'north_sp06', 'north_l_sp06', 'north_d_sp06', 'north_sp07', 'north_l_sp07', 'north_d_sp07', 'north_sp08', 'north_l_sp08', 'north_d_sp08', 'north_sp09', 'north_l_sp09', 'north_d_sp09', 'north_sp10', 'north_l_sp10', 'north_d_sp10', 'north_sp11', 'north_l_sp11', 'north_d_sp11', 'south_sp01', 'south

In [31]:
# define a list of column header basal positions
pos_list = ["cent", "north", "south", "ne", "se", "nw", "sw"]

In [32]:
def add_columns(df2, row):
    
    """ This function transfers iterrow data to output df. """
    
    df2['site'] = row['site']
    df2['uid'] =  row['uid']
    df2['date'] = str(row['am_date'])
    df2['factor'] = row['factor']
    df2['loc_count'] = row['count']
    
#     # replace 9999.0 from factor
#     fact_ = row['factor']
    
    
#     if fact_ == 9999.0:
#         print("fact_: ", fact_)
#         factor_ = 0.0
#         print("factor_: ", factor_)  
#     else:
#         factor_ == fact_
      
#     df2['factor'] = factor_
#     df2['factor'] = row['factor']
    df2['geometry'] = row['geometry']
    df2['lon_gda94'] = row['lon_gda94']
    df2['lat_gda94'] = row['lat_gda94']
    
    return df2

In [33]:
clean_gdf.columns
clean_gdf.shape

(120, 246)

In [34]:
import numpy as np
clean_gdf.fillna(9999, inplace=True)

In [35]:
def prop(df):
    
    """ This function calculates site proportions and returns df with alive proportions 
    ‘alv_prop’, dead proportions ‘ded_prop’ and total proportions ‘total_prop’. """
    
    print("--------------------------------------PROPORTIONS-----------------------------------")
    print(df)
    al_prop = []
    ded_prop = []
    total_prop = []
    tot_alive = df.alive.sum()
    print("tot_alive: ", tot_alive)

    tot_dead = df.dead.sum()
    print("tot_dead: ", tot_dead)
    tot_all = tot_alive + tot_dead
    print("tot_all: ", tot_all)
    for index, row in df.iterrows():
        print("-"*50)
        print("alive proportion: ", (row["alive"]/tot_alive)*100)
        al_prop.append((row["alive"]/tot_alive)*100)
        print("dead proportion: ", (row["dead"]/tot_dead)*100)
        ded_prop.append((row["dead"]/tot_dead)*100)
        print("total proportion: ", ((row["alive"] + row["dead"])/(tot_alive + tot_dead)*100))
        
        total_prop.append((row["alive"] + row["dead"])/(tot_alive + tot_dead)*100)
        
    df["alv_prop"] = al_prop
    df["ded_prop"] = ded_prop
    df["total_prop"] = total_prop
    
    return df
        

In [36]:
df_list = []
index_list = []

site_name_list = []
site_count_list = []
site_average_alive_list = []
site_average_dead_list = []
site_average_total_list = []

for index, row in clean_gdf.iterrows():
    print("index: ", index)
    print("row: ", row)
    row.dropna(inplace=True)
    site = row["site"]
    count = row["count"]
    uid = row["uid"]
    print("="*50)
    print(site)
    print("count: ", count)
    site_list = []
    index_list.append(site_list)
    species = []
    alive_count = []
    dead_count = []
    
    site_list = []
    count_list = []
    site_pos_al = []
    site_pos_ded = []
    site_pos_tot = []
    
    for pos in pos_list:
        pos_species = []
        pos_sp_live = []
        pos_sp_dead = []
        position_list = []
        
        pos_cols = [col for col in clean_df.columns if pos in col]
#         print("pos_cols: ", pos_cols)
        for cols in pos_cols:
#             print("="*50)
#             print(cols)
            if "{0}_sp".format(pos) in cols:
#                 print("species")

                if(row[cols] == "9999"):
                    pass
                    #print('true')
                elif(row[cols] == 9999):
                    pass
                    #print('true')
                elif(row[cols] == 9999.0):
                    pass
                    #print('true')
                else:
                    print("Valid species: ", row[cols])
                    species.append(row[cols])
                    pos_species.append(row[cols])
                    
                    
                                        
            elif "{0}_l".format(pos) in cols:
                #print("live")
                live = row[cols]
                
                if(row[cols] == "9999"):
                    pass
                    #print('true')
                elif(row[cols] == 9999):
                    pass
                    #print('true')
                elif(row[cols] == 9999.0):
                    pass
                    #print('true')
                else:
                    print("Valid alive: ", row[cols])
                    alive_count.append(row[cols])
                    pos_sp_live.append(row[cols])
                    
                
                    
            elif "{0}_d".format(pos) in cols:
                #print("dead")
                dead = row[cols]
                
                
                if(row[cols] == "9999"):
                    pass
                    #print('true')
                elif(row[cols] == 9999):
                    pass
                    #print('true')
                elif(row[cols] == 9999.0):
                    pass
                    #print('true')
                else:
                    print("Valid dead: ", row[cols])
                    dead_count.append(row[cols])
                    pos_sp_dead.append(row[cols])
                    
            else:
                print("ERROR"*100)

            
        sum_al = sum(pos_sp_live)
        sum_dead = sum(pos_sp_dead)
        sum_all = sum_al + sum_dead
        #print("sum_all: ", sum_all)
        #print(site)
        #print(count)
        site_list.append(site)
        count_list.append(count)
        site_pos_al.append(sum_al)
        site_pos_ded.append(sum_dead)
        site_pos_tot.append(sum_all)

        print("position totals: ", sum_al, sum_dead, sum_all)
        print("-"*50)
    print("Total basal count locations within site: ", count)
    print("List of ALIVE position totals stems: ", site_pos_al)
    print("Totals at: ", sum(site_pos_al))
    print("Equals: ", sum(site_pos_al)/count)


    print("List of DEAD position totals stems: ", site_pos_ded)
    print("Totals at: ", sum(site_pos_ded))
    print("Equals: ", sum(site_pos_ded)/count)
    

    print("List of TOTAL position totals stems: ", site_pos_tot)
    print("Totals at: ", sum(site_pos_tot))
    print("Equals: ", sum(site_pos_tot)/count)
    
    site_name_list.append(site)
    site_count_list.append(count)
    site_average_alive_list.append(sum(site_pos_al)/count)
    site_average_dead_list.append(sum(site_pos_ded)/count)
    site_average_total_list.append(sum(site_pos_tot)/count)

    
    print('='*20)
    print("species list: ", species)
    print("alive_count: ", alive_count)
    
    df = pd.DataFrame(list(zip(species, alive_count, dead_count)),
           columns =['species', 'alive', 'dead'])
    
    if not df.empty:

        print("Groupby Species: ")
        print(df)
        # groupby species list
        df2 = df.groupby('species').sum()
        df3 = df2.reset_index()
        print("*"*50)
        print("groupby dataframe: ", df3)
        print("df columns: ", df3.columns)

        # calculate strand species proprtions
        df3 = prop(df3)
        print("-"*50, df3)
        # add columns and fill
        df3 = add_columns(df3, row)


        print("-"*50, df3)
        if df3.empty:
            print("+"*50)
            print("Data frame is Empty")

            empty_header_list = ["species", "alive", "dead", "alv_prop", "ded_prop", "total_prop"]
            #,  'site', 'uid', 'date', 'factor', 'loc_count', 'geometry', 'lon_gda94', 'lat_gda94']
            empty_data_list = ["None", 0.0, 0.0, 0.0, 0.0, 0.0] #, site, uid, date, factor, loc_count, geometry, lon_gda94, lat_gda_94]

            # convert the list into dataframe row
            data = pd.DataFrame(empty_data_list).T

            # add columns
            data.columns = empty_header_list

            df3 = data
            df3 = add_columns(df3, row)

            print("+"*50)
            print("newdf3: ", df3)
        else:
            print("*"*50)
            print("Data frame is NOT Empty")
            df3 = add_columns(df3, row)
            df3_columns = df3.columns.tolist()
            print(df3_columns)
            print("*"*50) 

        df_list.append(df3)


# site_name_list = []
# site_count_list = []
# site_average_alive_list = []
# site_average_dead_list = []
# site_average_total_list = []

print("+"*50)
print(site_name_list)
print(site_average_alive_list)

site_average_alive_list
site_average_dead_list
site_average_total_list
print("-"*50)

# create lists of usefull data
factor_list = clean_df['factor'].tolist()
print("Factor list: ", factor_list)

factor = [0.0 if item == 9999.0 else item for item in factor_list]
print("Factor: ", factor)


## Convert averages by timing the site average strand counts by the basl wedge factor
cor_site_average_alive_list = [i*fac for i, fac in zip(site_average_alive_list, factor)]
cor_site_average_dead_list = [i*fac for i, fac in zip(site_average_dead_list, factor)]
cor_site_average_total_list = [i*fac for i, fac in zip(site_average_total_list, factor)]


df_av = pd.DataFrame(list(zip(site_name_list, factor, site_count_list, site_average_alive_list, site_average_dead_list, site_average_total_list,
                             cor_site_average_alive_list, cor_site_average_dead_list, cor_site_average_total_list)),
       columns =['site', 'factor', 'count', 'avg_alive', 'avg_dead', 'avg_total', 'cor_av_liv', 'cor_av_ded', 'cor_av_tot'])
print("DF_AV", df_av)

index:  0
row:  site                                                lit01
date_er                                        26/04/2013
date                                  2013-04-26 00:00:00
am_date                                          20130426
datum                                               GDA94
                                 ...                     
nw_d_sp11                                          9999.0
uid                                                     1
geometry     POINT (131.0619172006217 -12.54791228210612)
lon_gda94                                      131.061917
lat_gda94                                      -12.547912
Name: 0, Length: 246, dtype: object
lit01
count:  7
Valid species:  E. tetrodonta
Valid alive:  15.0
Valid dead:  0.0
Valid species:  E. chlorostachys
Valid alive:  3.0
Valid dead:  0.0
Valid species:  P. spiralis
Valid alive:  2.0
Valid dead:  0.0
Valid species:  C. armstrongii
Valid alive:  1.0
Valid dead:  0.0
Valid species:  Buchorvet
Valid

Valid dead:  0.0
Valid species:  Excoecaria parvifolia
Valid alive:  4.0
Valid dead:  0.0
position totals:  11.0 0.0 11.0
--------------------------------------------------
Valid species:  Lysiphyllum cunninghamii
Valid alive:  4.0
Valid dead:  0.0
Valid species:  Acacia valida
Valid alive:  1.0
Valid dead:  0.0
Valid species:  Atalaya hemiglauca
Valid alive:  1.0
Valid dead:  0.0
Valid species:  Terminalia volucris
Valid alive:  1.0
Valid dead:  0.0
Valid species:  Carissa lanceolata
Valid alive:  2.0
Valid dead:  0.0
position totals:  9.0 0.0 9.0
--------------------------------------------------
Valid species:  Lysiphyllum cunninghamii
Valid alive:  5.0
Valid dead:  0.0
Valid species:  Acacia valida
Valid alive:  6.0
Valid dead:  0.0
Valid species:  Atalaya hemiglauca
Valid alive:  2.0
Valid dead:  0.0
Valid species:  Terminalia volucris
Valid alive:  1.0
Valid dead:  0.0
position totals:  14.0 0.0 14.0
--------------------------------------------------
Total basal count locations w

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars



position totals:  12.0 11.0 23.0
--------------------------------------------------
Total basal count locations within site:  7
List of ALIVE position totals stems:  [15.0, 23.0, 14.0, 19.0, 13.0, 17.0, 12.0]
Totals at:  113.0
Equals:  16.142857142857142
List of DEAD position totals stems:  [3.0, 12.0, 9.0, 9.0, 6.0, 15.0, 11.0]
Totals at:  65.0
Equals:  9.285714285714286
List of TOTAL position totals stems:  [18.0, 35.0, 23.0, 28.0, 19.0, 32.0, 23.0]
Totals at:  178.0
Equals:  25.428571428571427
species list:  ['Acacia shirleyi', 'Acacia shirleyi', 'Acacia shirleyi', 'Macropteranthes kekwickii', 'Eucalyptus microtheca', 'Acacia shirleyi', 'Macropteranthes kekwickii', 'Acacia shirleyi', 'Acacia shirleyi', 'Acacia shirleyi', 'Macropteranthes kekwickii']
alive_count:  [15.0, 23.0, 12.0, 0.0, 2.0, 14.0, 5.0, 13.0, 17.0, 11.0, 1.0]
Groupby Species: 
                      species  alive  dead
0             Acacia shirleyi   15.0   3.0
1             Acacia shirleyi   23.0  12.0
2           

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


...

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: i

                species  alive  dead  alv_prop  ded_prop  total_prop   site  \
0  Terminalia arostrata    1.0   0.0     100.0       NaN       100.0  vrd16   

   uid      date  factor  loc_count  \
0   64  20120727     0.1          7   

                                      geometry   lon_gda94  lat_gda94  
0  POINT (130.8824382767118 -17.2742162119834)  130.882438 -17.274216  
**************************************************
Data frame is NOT Empty
['species', 'alive', 'dead', 'alv_prop', 'ded_prop', 'total_prop', 'site', 'uid', 'date', 'factor', 'loc_count', 'geometry', 'lon_gda94', 'lat_gda94']
**************************************************
index:  64
row:  site                                                vrd17
date_er                               2012-07-27 00:00:00
date                                  2012-07-27 00:00:00
am_date                                          20120727
datum                                               GDA94
                                 .

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: i

              species  alive  dead  alv_prop  ded_prop  total_prop   site  \
0  Atalaya hemiglauca    1.0   0.0      25.0       NaN        25.0  vrd32   
1  Calotropis procera    3.0   0.0      75.0       NaN        75.0  vrd32   

   uid      date  factor  loc_count  \
0   88  20120801     0.1          7   
1   88  20120801     0.1          7   

                                       geometry   lon_gda94  lat_gda94  
0  POINT (131.2389273242553 -16.20861252510795)  131.238927 -16.208613  
1  POINT (131.2389273242553 -16.20861252510795)  131.238927 -16.208613  
**************************************************
Data frame is NOT Empty
['species', 'alive', 'dead', 'alv_prop', 'ded_prop', 'total_prop', 'site', 'uid', 'date', 'factor', 'loc_count', 'geometry', 'lon_gda94', 'lat_gda94']
**************************************************
index:  88
row:  site                                                vrd33
date_er                               2012-08-01 00:00:00
date                 

                   species  alive  dead
0            E. tetrodonta   13.0   1.0
1   Corymbia dichromphloia    7.0   1.0
2            E. tetrodonta   11.0   1.0
3   Corymbia dichromphloia    5.0   0.0
4       Buchanania obovata    2.0   0.0
5            E. tetrodonta    9.0   0.0
6   Corymbia dichromphloia    5.0   0.0
7         E. chlorostachys    3.0   0.0
8            E. tetrodonta   11.0   0.0
9   Corymbia dichromphloia    9.0   0.0
10        E. chlorostachys    3.0   0.0
11      Buchanania obovata    1.0   0.0
12           E. tetrodonta    8.0   1.0
13  Corymbia dichromphloia    9.0   0.0
14        E. chlorostachys    1.0   1.0
15           E. tetrodonta   10.0   1.0
16  Corymbia dichromphloia    5.0   3.0
17        E. chlorostachys    1.0   0.0
18           E. tetrodonta   11.0   1.0
19  Corymbia dichromphloia    9.0   0.0
20      Buchanania obovata    1.0   0.0
**************************************************
groupby dataframe:                    species  alive  dead
0      Buc

# Site proportion totals

Dataframe df_av contains some basal data where the factor was not recorded. These sites need to be removed.

In [37]:
print(df_av.shape)
print(len(df_av.site.unique()))

(120, 9)
120


In [38]:
df_av

,site,factor,count,avg_alive,avg_dead,avg_total,cor_av_liv,cor_av_ded,cor_av_tot
0,lit01,0.50,7,34.571429,1.428571,36.000000,17.285714,0.714286,18.000000
1,girra02,0.10,7,20.714286,3.428571,24.142857,2.071429,0.342857,2.414286
2,umb07,0.25,7,19.428571,1.857143,21.285714,4.857143,0.464286,5.321429
3,jdr01,0.75,7,23.285714,0.142857,23.428571,17.464286,0.107143,17.571429
4,jdr02,0.50,7,27.428571,1.142857,28.571429,13.714286,0.571429,14.285714
...,...,...,...,...,...,...,...,...,...
115,ant09,0.00,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,ant07,0.00,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,ant06,0.25,7,16.428571,2.142857,18.571429,4.107143,0.535714,4.642857
118,barkely02,0.00,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
df_av.to_csv(os.path.join(output_dir, "all_slats_site_proportions.csv"), index=False)

In [40]:
print("There are", df_av.shape[0], "obsevations.")
# filter df for data where avg_tot is greater than 0 and 
gr0_fac0 = df_av[(df_av["avg_total"] > 0) & (df_av['factor']==0)]
print("With" , gr0_fac0.shape[0], "missing factors")
#df_av.to_csv(os.path.join(output_dir, "slats_site_prop_fac0_avg_tot_gr0.csv"), index=False)

# drop those rows from the dataframe 
drop_index = gr0_fac0.index.tolist()

df_av.drop(df_av.index[drop_index], inplace=True)
# df = df_av[~df_av.isin(gr0_fac0)].dropna(inplace=True)
df_av.to_csv(os.path.join(output_dir, "clean_slats_site_proportions.csv"), index=False)
print(df_av.shape)

There are 120 obsevations.
With 16 missing factors
(104, 9)


The following sites were dropped due to a missing factor value:

In [41]:
print(gr0_fac0.site)

11    auv04
24    amg02
29    amg09
32    ant05
48    lim02
51    lim05
64     wh01
73     wh10
74     wh11
75     wh12
77     wh14
81    vrd18
82    vrd19
84    vrd21
90    vrd27
91    vrd28
Name: site, dtype: object


In [42]:
for i in df_list:
    print("-"*50)
    print(i)

--------------------------------------------------
                   species  alive  dead   alv_prop  ded_prop  total_prop  \
0                Buchorvet   11.0   0.0   4.545455       0.0    4.365079   
1           C. armstrongii    3.0   0.0   1.239669       0.0    1.190476   
2               C. fraseri    2.0   0.0   0.826446       0.0    0.793651   
3             Corymbia sp.    1.0   0.0   0.413223       0.0    0.396825   
4         E. chlorostachys   38.0   2.0  15.702479      20.0   15.873016   
5               E. miniata   18.0   0.0   7.438017       0.0    7.142857   
6            E. tetrodonta   98.0   4.0  40.495868      40.0   40.476190   
7              F. achleata    3.0   0.0   1.239669       0.0    1.190476   
8               L. humilis    5.0   0.0   2.066116       0.0    1.984127   
9                P. careya    6.0   0.0   2.479339       0.0    2.380952   
10            P. pubescens    1.0   0.0   0.413223       0.0    0.396825   
11             P. spiralis   48.0   4

                species  alive  dead   alv_prop   ded_prop  total_prop  \
0         Antidesma sp.    7.0   0.0   4.242424   0.000000    3.932584   
1    Buchanania obovata    6.0   0.0   3.636364   0.000000    3.370787   
2      E. chlorostachys    0.0   1.0   0.000000   7.692308    0.561798   
3         E. tetrodonta  125.0   8.0  75.757576  61.538462   74.719101   
4  Exocarpos latifolius    5.0   0.0   3.030303   0.000000    2.808989   
5          Gardenia sp.   11.0   0.0   6.666667   0.000000    6.179775   
6            L. humilis    3.0   0.0   1.818182   0.000000    1.685393   
7             P. careya    1.0   0.0   0.606061   0.000000    0.561798   
8      Strychnos lucida    2.0   0.0   1.212121   0.000000    1.123596   
9        Terminalia sp.    5.0   4.0   3.030303  30.769231    5.056180   

     site  uid      date  factor  loc_count  \
0  buff01    3  20210713    0.25          7   
1  buff01    3  20210713    0.25          7   
2  buff01    3  20210713    0.25          7 

                 species  alive  dead   alv_prop  ded_prop  total_prop   site  \
0     Acacia holosericea    1.0   0.0   0.751880       0.0    0.689655  vrd30   
1  Bauhinia cunninghamii    1.0   0.0   0.751880       0.0    0.689655  vrd30   
2   Eucalyptus tectifica  122.0  12.0  91.729323     100.0   92.413793  vrd30   
3   Exocarpos parvifolia    9.0   0.0   6.766917       0.0    6.206897  vrd30   

   uid      date  factor  loc_count  \
0   86  20120731    0.25          7   
1   86  20120731    0.25          7   
2   86  20120731    0.25          7   
3   86  20120731    0.25          7   

                                       geometry   lon_gda94  lat_gda94  
0  POINT (131.1141672006653 -16.33828722537855)  131.114167 -16.338287  
1  POINT (131.1141672006653 -16.33828722537855)  131.114167 -16.338287  
2  POINT (131.1141672006653 -16.33828722537855)  131.114167 -16.338287  
3  POINT (131.1141672006653 -16.33828722537855)  131.114167 -16.338287  
---------------------------------

                    species  alive  dead   alv_prop   ded_prop  total_prop  \
0                Acacia sp.   17.0   4.0   8.629442  33.333333   10.047847   
1        Buchanania obovata   11.0   0.0   5.583756   0.000000    5.263158   
2                E. miniata   92.0   8.0  46.700508  66.666667   47.846890   
3             E. tetrodonta   26.0   0.0  13.197970   0.000000   12.440191   
4       Grevillea agrifolia    1.0   0.0   0.507614   0.000000    0.478469   
5                Owinea sp.    1.0   0.0   0.507614   0.000000    0.478469   
6                 P. careya   17.0   0.0   8.629442   0.000000    8.133971   
7              P. pubescens    3.0   0.0   1.522843   0.000000    1.435407   
8         Persoonia falcata    2.0   0.0   1.015228   0.000000    0.956938   
9    Petalostigma pubescens   16.0   0.0   8.121827   0.000000    7.655502   
10  Terminalia carpentariae   11.0   0.0   5.583756   0.000000    5.263158   

      site  uid      date  factor  loc_count  \
0   legu07  110

In [43]:
df1 = pd.concat(df_list)
df2 = df1.reset_index(drop=True)
df2.shape

(356, 14)

In [44]:
df2

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,loc_count,geometry,lon_gda94,lat_gda94
0,Buchorvet,11.0,0.0,4.545455,0.000000,4.365079,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
1,C. armstrongii,3.0,0.0,1.239669,0.000000,1.190476,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
2,C. fraseri,2.0,0.0,0.826446,0.000000,0.793651,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
3,Corymbia sp.,1.0,0.0,0.413223,0.000000,0.396825,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
4,E. chlorostachys,38.0,2.0,15.702479,20.000000,15.873016,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Atalaya hemiglauca,4.0,0.0,3.669725,0.000000,3.030303,el02,94,20130414,0.1,7,POINT (133.528790900227 -17.52949263005511),133.528791,-17.529493
352,Bauhinia cunninghamii,84.0,20.0,77.064220,86.956522,78.787879,el02,94,20130414,0.1,7,POINT (133.528790900227 -17.52949263005511),133.528791,-17.529493
353,Corymbia dichromphloia,9.0,3.0,8.256881,13.043478,9.090909,el02,94,20130414,0.1,7,POINT (133.528790900227 -17.52949263005511),133.528791,-17.529493
354,Corymbia terminalis,1.0,0.0,0.917431,0.000000,0.757576,el02,94,20130414,0.1,7,POINT (133.528790900227 -17.52949263005511),133.528791,-17.529493


In [45]:

df2['factor'] = df2['factor'].replace(9999.0,0.0)
print(df2.shape)
#print(df2)


(356, 14)


In [46]:
df_list[0]
print(len(df_list))

72


In [47]:
df2.head(10)

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,loc_count,geometry,lon_gda94,lat_gda94
0,Buchorvet,11.0,0.0,4.545455,0.0,4.365079,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
1,C. armstrongii,3.0,0.0,1.239669,0.0,1.190476,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
2,C. fraseri,2.0,0.0,0.826446,0.0,0.793651,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
3,Corymbia sp.,1.0,0.0,0.413223,0.0,0.396825,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
4,E. chlorostachys,38.0,2.0,15.702479,20.0,15.873016,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
5,E. miniata,18.0,0.0,7.438017,0.0,7.142857,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
6,E. tetrodonta,98.0,4.0,40.495868,40.0,40.476190,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
7,F. achleata,3.0,0.0,1.239669,0.0,1.190476,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
8,L. humilis,5.0,0.0,2.066116,0.0,1.984127,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912
9,P. careya,6.0,0.0,2.479339,0.0,2.380952,lit01,1,20130426,0.5,7,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912


In [48]:
print(df2.shape)
print(len(df2.site.unique()))

(356, 14)
72


In [49]:
df_av.shape

(104, 9)

In [50]:
print(sorted(df_av.site.unique()))

['agb02', 'amg01', 'amg03', 'amg05', 'amg06', 'amg08', 'amg10', 'ant01', 'ant02', 'ant03', 'ant04', 'ant06', 'ant07', 'ant09', 'antgr01', 'auv01', 'auv02', 'auv03', 'auv05', 'auv06', 'auv07', 'auv08', 'auv09', 'barkely01', 'barkely02', 'barkely03', 'barkely04', 'barkely05', 'barkely06', 'barkely07', 'buff01', 'centa13', 'centa14', 'el01', 'el02', 'ep01', 'girra01', 'girra02', 'gulf10', 'gulf11', 'gulf12', 'hsf01', 'hsf02', 'hshr01', 'jdr01', 'jdr02', 'jdr03', 'jdr04', 'jdr05', 'larr15', 'legu01', 'legu02', 'legu03', 'legu04', 'legu05', 'legu06', 'legu07', 'legu08', 'lim01', 'lim03', 'lim04', 'lim06', 'lim08', 'lim09', 'lit01', 'lto04', 'lto05', 'nc01', 'nc02', 'nc03', 'nt001', 'nt002', 'nt003', 'nt004', 'umb07', 'vrd16', 'vrd17', 'vrd20', 'vrd22', 'vrd23', 'vrd24', 'vrd25', 'vrd26', 'vrd29', 'vrd30', 'vrd31', 'vrd32', 'vrd33', 'vrd34', 'vrd35', 'vrd36', 'vrd37', 'wedo01', 'wedo03', 'wh02', 'wh03', 'wh04', 'wh05', 'wh06', 'wh07', 'wh08', 'wh09', 'wh13', 'wh15']


In [51]:
df_av.factor.unique()

array([0.5 , 0.1 , 0.25, 0.75, 0.  , 1.  ])

In [52]:
print(sorted(df2.site.unique()))

['amg01', 'amg02', 'amg05', 'amg06', 'amg08', 'amg09', 'amg10', 'ant03', 'ant05', 'ant06', 'auv01', 'auv02', 'auv03', 'auv04', 'auv05', 'barkely07', 'buff01', 'centa13', 'centa14', 'el02', 'ep01', 'girra01', 'girra02', 'gulf10', 'gulf11', 'hsf01', 'hsf02', 'hshr01', 'jdr01', 'jdr02', 'jdr03', 'jdr04', 'legu01', 'legu04', 'legu05', 'legu06', 'legu07', 'legu08', 'lim01', 'lim02', 'lim03', 'lim04', 'lim05', 'lim06', 'lit01', 'nc01', 'nc02', 'nc03', 'umb07', 'vrd16', 'vrd18', 'vrd19', 'vrd20', 'vrd21', 'vrd22', 'vrd23', 'vrd25', 'vrd27', 'vrd28', 'vrd30', 'vrd32', 'vrd33', 'vrd37', 'wedo01', 'wedo03', 'wh01', 'wh02', 'wh07', 'wh10', 'wh11', 'wh12', 'wh14']


In [53]:
df2.factor.unique()

array([0.5 , 0.1 , 0.25, 0.75, 0.  , 1.  ])

In [54]:
df2.to_csv(os.path.join(output_dir, "df2_v6.csv"), index=False)

In [55]:
#result = pd.merge(df2, df_av, on=["site", "site"])

In [56]:
result = pd.merge(df2, df_av, on=["site", "factor"], how="outer")
result.to_csv(os.path.join(output_dir, "result_outer.csv"), index=False)

In [57]:
result.shape

(404, 21)

In [58]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404 entries, 0 to 403
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   species     356 non-null    object 
 1   alive       356 non-null    float64
 2   dead        356 non-null    float64
 3   alv_prop    356 non-null    float64
 4   ded_prop    316 non-null    float64
 5   total_prop  356 non-null    float64
 6   site        404 non-null    object 
 7   uid         356 non-null    float64
 8   date        356 non-null    object 
 9   factor      404 non-null    float64
 10  loc_count   356 non-null    float64
 11  geometry    356 non-null    object 
 12  lon_gda94   356 non-null    float64
 13  lat_gda94   356 non-null    float64
 14  count       353 non-null    float64
 15  avg_alive   353 non-null    float64
 16  avg_dead    353 non-null    float64
 17  avg_total   353 non-null    float64
 18  cor_av_liv  353 non-null    float64
 19  cor_av_ded  353 non-null    f

In [59]:
result.species.fillna("No basal", inplace=True)

In [60]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404 entries, 0 to 403
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   species     404 non-null    object 
 1   alive       356 non-null    float64
 2   dead        356 non-null    float64
 3   alv_prop    356 non-null    float64
 4   ded_prop    316 non-null    float64
 5   total_prop  356 non-null    float64
 6   site        404 non-null    object 
 7   uid         356 non-null    float64
 8   date        356 non-null    object 
 9   factor      404 non-null    float64
 10  loc_count   356 non-null    float64
 11  geometry    356 non-null    object 
 12  lon_gda94   356 non-null    float64
 13  lat_gda94   356 non-null    float64
 14  count       353 non-null    float64
 15  avg_alive   353 non-null    float64
 16  avg_dead    353 non-null    float64
 17  avg_total   353 non-null    float64
 18  cor_av_liv  353 non-null    float64
 19  cor_av_ded  353 non-null    f

## Fill missing values of no basal data

In [61]:
# select out no basal
no_basal = result[result["species"] == "No basal"]
no_basal.fillna(0.0, inplace = True)

C:\Users\rmcgr\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [62]:
no_basal

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,geometry,lon_gda94,lat_gda94,count,avg_alive,avg_dead,avg_total,cor_av_liv,cor_av_ded,cor_av_tot
356,No basal,0.0,0.0,0.0,0.0,0.0,jdr05,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
357,No basal,0.0,0.0,0.0,0.0,0.0,auv06,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
358,No basal,0.0,0.0,0.0,0.0,0.0,auv07,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
359,No basal,0.0,0.0,0.0,0.0,0.0,auv08,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
360,No basal,0.0,0.0,0.0,0.0,0.0,auv09,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
361,No basal,0.0,0.0,0.0,0.0,0.0,nt001,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
362,No basal,0.0,0.0,0.0,0.0,0.0,nt002,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
363,No basal,0.0,0.0,0.0,0.0,0.0,nt003,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
364,No basal,0.0,0.0,0.0,0.0,0.0,nt004,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
365,No basal,0.0,0.0,0.0,0.0,0.0,amg03,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
df_missing = clean_df[["site", "am_date", "lon_gda94", "lat_gda94", "geometry"]]
df_missing.reset_index(inplace=True)

In [64]:
cleaned = pd.merge(no_basal, df_missing, on=["site"], how="inner")

In [65]:
cleaned

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid_x,date,factor,...,avg_dead,avg_total,cor_av_liv,cor_av_ded,cor_av_tot,uid_y,am_date,lon_gda94_y,lat_gda94_y,geometry_y
0,No basal,0.0,0.0,0.0,0.0,0.0,jdr05,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,115,20120524,131.625270,-14.111267,POINT (131.62527 -14.11127)
1,No basal,0.0,0.0,0.0,0.0,0.0,auv06,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,121,20121011,129.685010,-16.007755,POINT (129.68501 -16.00775)
2,No basal,0.0,0.0,0.0,0.0,0.0,auv07,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,121,20121011,129.690385,-16.070366,POINT (129.69039 -16.07037)
3,No basal,0.0,0.0,0.0,0.0,0.0,auv08,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,122,20121011,129.624640,-16.198472,POINT (129.62464 -16.19847)
4,No basal,0.0,0.0,0.0,0.0,0.0,auv09,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,123,20121011,129.648024,-16.162632,POINT (129.64802 -16.16263)
5,No basal,0.0,0.0,0.0,0.0,0.0,nt001,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,124,20110523,131.209577,-13.945196,POINT (131.20958 -13.94520)
6,No basal,0.0,0.0,0.0,0.0,0.0,nt002,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125,20120523,131.218327,-13.934318,POINT (131.21833 -13.93432)
7,No basal,0.0,0.0,0.0,0.0,0.0,nt003,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,126,20120523,131.182433,-13.846585,POINT (131.18243 -13.84658)
8,No basal,0.0,0.0,0.0,0.0,0.0,nt004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,127,20120524,130.860744,-13.058875,POINT (130.86074 -13.05888)
9,No basal,0.0,0.0,0.0,0.0,0.0,amg03,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,26,20120620,134.031626,-16.778991,POINT (134.03163 -16.77899)


In [66]:
cleaned["uid_x"] = cleaned["uid_y"]
cleaned["date"] = cleaned["am_date"]
cleaned["lon_gda94_x"] = cleaned["lon_gda94_y"]
cleaned["lat_gda94_x"] = cleaned["lat_gda94_y"]
cleaned["geometry_x"] = cleaned["geometry_y"]

In [67]:
cleaned.drop(["uid_y", "am_date", "lon_gda94_y", "lat_gda94_y", "geometry_y"], axis=1, inplace=True)

In [68]:
dict_ = {"uid_x": "uid",
        "lon_gda94_x": "lon_gda94",
        "lat_gda94_x": "lat_gda94",
        "geometry_x": "geometry"}
 
# call rename () method
cleaned.rename(columns=dict_,
          inplace=True)

In [69]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   species     48 non-null     object  
 1   alive       48 non-null     float64 
 2   dead        48 non-null     float64 
 3   alv_prop    48 non-null     float64 
 4   ded_prop    48 non-null     float64 
 5   total_prop  48 non-null     float64 
 6   site        48 non-null     object  
 7   uid         48 non-null     int64   
 8   date        48 non-null     object  
 9   factor      48 non-null     float64 
 10  loc_count   48 non-null     float64 
 11  geometry    48 non-null     geometry
 12  lon_gda94   48 non-null     float64 
 13  lat_gda94   48 non-null     float64 
 14  count       48 non-null     float64 
 15  avg_alive   48 non-null     float64 
 16  avg_dead    48 non-null     float64 
 17  avg_total   48 non-null     float64 
 18  cor_av_liv  48 non-null     float64 
 19  cor_av_ded

In [70]:
print(sorted(cleaned.site.unique()))
print(cleaned.shape)

['agb02', 'amg03', 'ant01', 'ant02', 'ant04', 'ant07', 'ant09', 'antgr01', 'auv06', 'auv07', 'auv08', 'auv09', 'barkely01', 'barkely02', 'barkely03', 'barkely04', 'barkely05', 'barkely06', 'el01', 'gulf12', 'jdr05', 'larr15', 'legu02', 'legu03', 'lim08', 'lim09', 'lto04', 'lto05', 'nt001', 'nt002', 'nt003', 'nt004', 'vrd17', 'vrd24', 'vrd26', 'vrd29', 'vrd31', 'vrd34', 'vrd35', 'vrd36', 'wh03', 'wh04', 'wh05', 'wh06', 'wh08', 'wh09', 'wh13', 'wh15']
(48, 21)


In [71]:
w_fact = result[(result["avg_total"] >0) & (result["species"] != "No basal")]

In [72]:
print(sorted(w_fact.site.unique()))
w_fact.shape

['amg01', 'amg05', 'amg06', 'amg08', 'amg10', 'ant03', 'ant06', 'auv01', 'auv02', 'auv03', 'auv05', 'barkely07', 'buff01', 'centa13', 'centa14', 'el02', 'ep01', 'girra01', 'girra02', 'gulf10', 'gulf11', 'hsf01', 'hsf02', 'hshr01', 'jdr01', 'jdr02', 'jdr03', 'jdr04', 'legu01', 'legu04', 'legu05', 'legu06', 'legu07', 'legu08', 'lim01', 'lim03', 'lim04', 'lim06', 'lit01', 'nc01', 'nc02', 'nc03', 'umb07', 'vrd16', 'vrd20', 'vrd22', 'vrd23', 'vrd25', 'vrd30', 'vrd32', 'vrd33', 'vrd37', 'wedo01', 'wedo03', 'wh02', 'wh07']


(305, 21)

In [73]:
df_cleanedup = pd.concat([w_fact, cleaned])

In [74]:
df_cleanedup.shape

(353, 21)

In [75]:
# print("There are", df_av.shape[0], "obsevations.")
# # filter df for data where avg_tot is greater than 0 and 
# gr0_fac0 = df_av[(df_av["avg_total"] > 0) & (df_av['factor']==0)]
# print("With" , gr0_fac0.shape[0], "missing factors")
# #df_av.to_csv(os.path.join(output_dir, "slats_site_prop_fac0_avg_tot_gr0.csv"), index=False)

# # drop those rows from the dataframe 
# drop_index = gr0_fac0.index.tolist()

# df_av.drop(df_av.index[drop_index], inplace=True)
# # df = df_av[~df_av.isin(gr0_fac0)].dropna(inplace=True)
# df_av.to_csv(os.path.join(output_dir, "clean_slats_site_proportions.csv"), index=False)
# print(df_av.shape)

### Dataframe has site averages and coorrected averages calculated

column headers:
 - avg: average stem counts per site (i.e. sum(cent, north etc.)/num(basal counts within site)
 - cor_av: corrected stem count average (i.e. avg * factor)
    

In [76]:
df_cleanedup

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,geometry,lon_gda94,lat_gda94,count,avg_alive,avg_dead,avg_total,cor_av_liv,cor_av_ded,cor_av_tot
0,Buchorvet,11.0,0.0,4.545455,0.0,4.365079,lit01,1.0,20130426,0.5,...,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912,7.0,34.571429,1.428571,36.0,17.285714,0.714286,18.0
1,C. armstrongii,3.0,0.0,1.239669,0.0,1.190476,lit01,1.0,20130426,0.5,...,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912,7.0,34.571429,1.428571,36.0,17.285714,0.714286,18.0
2,C. fraseri,2.0,0.0,0.826446,0.0,0.793651,lit01,1.0,20130426,0.5,...,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912,7.0,34.571429,1.428571,36.0,17.285714,0.714286,18.0
3,Corymbia sp.,1.0,0.0,0.413223,0.0,0.396825,lit01,1.0,20130426,0.5,...,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912,7.0,34.571429,1.428571,36.0,17.285714,0.714286,18.0
4,E. chlorostachys,38.0,2.0,15.702479,20.0,15.873016,lit01,1.0,20130426,0.5,...,POINT (131.0619172006217 -12.54791228210612),131.061917,-12.547912,7.0,34.571429,1.428571,36.0,17.285714,0.714286,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,No basal,0.0,0.0,0.000000,0.0,0.000000,lim08,17.0,20130601,0.0,...,POINT (135.3050934998249 -15.23416616978125),135.305093,-15.234166,7.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
44,No basal,0.0,0.0,0.000000,0.0,0.000000,lim09,18.0,20130601,0.0,...,POINT (135.3548298002856 -15.37926174966222),135.354830,-15.379262,7.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
45,No basal,0.0,0.0,0.000000,0.0,0.000000,ant09,39.0,20120626,0.0,...,POINT (135.2698067994025 -18.32963670886927),135.269807,-18.329637,7.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
46,No basal,0.0,0.0,0.000000,0.0,0.000000,ant07,38.0,20120626,0.0,...,POINT (135.2038401986589 -18.24579776721202),135.203840,-18.245798,7.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [77]:
print(len(df_cleanedup.site.unique()))

104


In [78]:
#df2_ = basal_area_m2(result)
df2_ = df_cleanedup

In [79]:
df2_.sample(3)

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,geometry,lon_gda94,lat_gda94,count,avg_alive,avg_dead,avg_total,cor_av_liv,cor_av_ded,cor_av_tot
113,dead,3.0,0.0,2.127660,0.00,1.923077,amg10,32.0,20120622,0.10,...,POINT (134.1876887644216 -16.41557846738749),134.187689,-16.415578,7.0,20.142857,2.142857,22.285714,2.014286,0.214286,2.228571
250,xanta dara,2.0,1.0,0.995025,6.25,1.382488,wedo03,11.0,20120710,0.25,...,POINT (130.9831230543892 -12.5339548490209),130.983123,-12.533955,7.0,28.714286,2.285714,31.000000,7.178571,0.571429,7.750000
229,Buchanania obovata,1.0,0.0,0.452489,0.00,0.420168,wedo01,10.0,20120606,0.10,...,POINT (131.0050186100947 -12.60400827072699),131.005019,-12.604008,7.0,31.571429,2.428571,34.000000,3.157143,0.242857,3.400000


In [101]:
df2_.to_csv(r"F:\cdu\data\output\20221221\test.csv", index = False)

In [81]:
def define_class(df):
    
    class_ = []
    leaves = []
    leaves_oth = []
    twigs = []
    bark = []
    bark_oth = []
    wood = []
    wood_oth = []
    branches = []
    branches_oth = []
    stems = []
    stems_oth = []
    agb = []
    agb_oth = []
    root = []
    root_oth = []
    tot_bm = []
    
    
    for index, row in df.iterrows():
        
        species = row["species"]
        print(species)
        
    
        if species == "E. tetrodonta" or species == " Eucalyptus tetrodonta":
            
            #print("Eute")
            class_.append("Eute")
            leaves.append(122)
            leaves_oth.append(0.84)
            twigs.append(127)
            bark.append(341)
            bark_oth.append(0.99)
            wood.append(2161)
            wood_oth.append(0.93)
            branches.append(799)
            branches_oth.append(0.85)
            stems.append(2502)
            stems_oth.append(0.95)
            agb.append(3403)
            agb_oth.append(0.97)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3945)
            
        elif species == "E. miniata" or species == "Eucalyptus miniata":
   
            #print("Eumi")
            class_.append("Eumi")
            leaves.append(50)
            leaves_oth.append(0.96)
            twigs.append(52)
            bark.append(218)
            bark_oth.append(0.92)
            wood.append(1829)
            wood_oth.append(0.95)
            branches.append(375)
            branches_oth.append(0.79)
            stems.append(2047)
            stems_oth.append(0.96)
            agb.append(2472)
            agb_oth.append(0.96)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3014)

        elif species == "E. porrecta" or species == "Eucalyptus porrecta" or species =="C. porrecta" or species == "Corymbia porrecta":
               
            #print("Eupo")    
            class_.append("Eupo")
            leaves.append(73)
            leaves_oth.append(0.85)
            twigs.append(76)
            bark.append(326)
            bark_oth.append(0.98)
            wood.append(1289)
            wood_oth.append(0.98)
            branches.append(619)
            branches_oth.append(0.96)
            stems.append(1616)
            stems_oth.append(0.90)
            agb.append(2308)
            agb_oth.append(0.98)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(2850)
            
        elif species == "E. bleeseri" or species == "Eucalyptus bleeseri" or species =="C. bleeseri" or species == "Corymbia bleeseri":
            
            #print("Eubl")
            class_.append("Eubl")
            leaves.append(49)
            leaves_oth.append(0.80)
            twigs.append(51)
            bark.append(347)
            bark_oth.append(0.97)
            wood.append(2225)
            wood_oth.append(0.97)
            branches.append(1163)
            branches_oth.append(0.90)
            stems.append(2573)
            stems_oth.append(0.97)
            agb.append(3785)
            agb_oth.append(0.96)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(4327)
            
            
        elif species == "E. chlorostachys" or species == "Erythrophleum chlorostachys":
            
            #print("Erch")
            class_.append("Erch")
            leaves.append(154)
            leaves_oth.append(0.75)
            twigs.append(160)
            bark.append(401)
            bark_oth.append(0.95)
            wood.append(1044)
            wood_oth.append(0.92)
            branches.append(814)
            branches_oth.append(0.62)
            stems.append(1445)
            stems_oth.append(0.93)
            agb.append(2413)
            agb_oth.append(0.82)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(2955)
            
        elif species == "No basal":
            
            #print("Erch")
            class_.append("Nobsl")
            leaves.append(0.0)
            leaves_oth.append(0.0)
            twigs.append(0.0)
            bark.append(0.0)
            bark_oth.append(0.0)
            wood.append(0.0)
            wood_oth.append(0.0)
            branches.append(0.0)
            branches_oth.append(0.0)
            stems.append(0.0)
            stems_oth.append(0.0)
            agb.append(0.0)
            agb_oth.append(0.0)
            root.append(0.0)
            root_oth.append(0.0)
            tot_bm.append(0.0)
            
            
        else:

            class_.append("Tefe")
            leaves.append(93)
            leaves_oth.append(0.89)
            twigs.append(97)
            bark.append(379)
            bark_oth.append(0.92)
            wood.append(1233)
            wood_oth.append(0.92)
            branches.append(935)
            branches_oth.append(0.83)
            stems.append(1612)
            stems_oth.append(0.92)
            agb.append(2640)
            agb_oth.append(0.91)
            root.append(542)
            root_oth.append(0.57)
            tot_bm.append(3182)
            
    df["co_ef_nme"] = class_
    df["leaves"] = leaves
    df["leaves_r2"] = leaves_oth
    df["twigs"] = twigs
    df["bark"] = bark
    df["bark_r2"] = bark_oth
    df["wood"] = wood
    df["wood_r2"] = wood_oth
    df["branches"]= branches
    df["branches_r2"] = branches_oth
    df["stems"] = stems
    df["stems_r2"] = stems_oth
    df["agb"] = agb
    df["agb_r2"] = agb_oth
    df["root"] = root
    df["root_r2"] = root_oth
    df["tot_bm"] = tot_bm
    
    
    return df
            
    

In [82]:
df3 = df2_

In [83]:
df4 = define_class(df3)
df4.to_csv(os.path.join(output_dir, "df4.csv"), index=False)
df4

Buchorvet
C. armstrongii
C. fraseri
Corymbia sp.
E. chlorostachys
E. miniata
E. tetrodonta
F. achleata
L. humilis
P. careya
P. pubescens
P. spiralis
Syzygium suborbiculare
Eucalyptus alba
Lophostemon lactifluus
Melaleuca nervoa
P. spiralis
Acacia latesens
Buchanania obovata
Corymbia ferruginea
Corymbia sp.
Denhmai obscura
E. chlorostachys
E. miniata
E. tetrodonta
Eucalyptus sp.
Exacarpos latafolia
Owinea sp.
Terminalia carpentariae
Corymbia sp.
E. miniata
Corymbia sp.
E. tetrodonta
Syzygium sp.
species 1
species 2
Atalaya hemiglauca
Eucalyptus microtheca
Lysiphyllum cunninghamii
Terminalia volucris
Acacia sp.
Brachychiton diversifolius
Corymbia confertiflora
Eucalyptus pruinosa
Eucalyptus terminalis
Lysiphyllum cunninghamii
Melaleuca minutifolia
Terminalia arostrata
Acacia valida
Atalaya hemiglauca
Carissa lanceolata
Excoecaria parvifolia
Lysiphyllum cunninghamii
Terminalia volucris
Lysiphyllum cunninghamii
Terminalia platyptera
Terminalia volucris
Buchanania obovata
Corymbia bleeseri


,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,wood_r2,branches,branches_r2,stems,stems_r2,agb,agb_r2,root,root_r2,tot_bm
0,Buchorvet,11.0,0.0,4.545455,0.0,4.365079,lit01,1.0,20130426,0.5,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
1,C. armstrongii,3.0,0.0,1.239669,0.0,1.190476,lit01,1.0,20130426,0.5,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
2,C. fraseri,2.0,0.0,0.826446,0.0,0.793651,lit01,1.0,20130426,0.5,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
3,Corymbia sp.,1.0,0.0,0.413223,0.0,0.396825,lit01,1.0,20130426,0.5,...,0.92,935.0,0.83,1612.0,0.92,2640.0,0.91,542.0,0.57,3182.0
4,E. chlorostachys,38.0,2.0,15.702479,20.0,15.873016,lit01,1.0,20130426,0.5,...,0.92,814.0,0.62,1445.0,0.93,2413.0,0.82,542.0,0.57,2955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,No basal,0.0,0.0,0.000000,0.0,0.000000,lim08,17.0,20130601,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
44,No basal,0.0,0.0,0.000000,0.0,0.000000,lim09,18.0,20130601,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
45,No basal,0.0,0.0,0.000000,0.0,0.000000,ant09,39.0,20120626,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
46,No basal,0.0,0.0,0.000000,0.0,0.000000,ant07,38.0,20120626,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [84]:
for i in df4.columns:
    print(i)

species
alive
dead
alv_prop
ded_prop
total_prop
site
uid
date
factor
loc_count
geometry
lon_gda94
lat_gda94
count
avg_alive
avg_dead
avg_total
cor_av_liv
cor_av_ded
cor_av_tot
co_ef_nme
leaves
leaves_r2
twigs
bark
bark_r2
wood
wood_r2
branches
branches_r2
stems
stems_r2
agb
agb_r2
root
root_r2
tot_bm


In [85]:
def total_ba_m2(df):
    
    # Total basal area
    tot_ba = []
    al_ba = []
    ded_ba = []
    
    # Carbom stock
    c_leaves = []
    c_twigs = []
    c_bark = []
    c_wood = []
    c_branches = []
    c_stems = []
    c_agb = []
    c_roots = []
    

        
    for index, row in df.iterrows():
        print("="*50)
        print("row: ", row)
        
        # Calculate total site basal area
        total_basal = (row["total_prop"] * row["cor_av_tot"])/100
        print("total basal: ", total_basal)
        tot_ba.append(total_basal)
        #tot_ba.append((row["total_prop"] * row["cor_av_tot"])/100)
        
        alive_basal = (row["alv_prop"] * row["cor_av_liv"])/100
        print("alive basal: ", alive_basal)
        al_ba.append(alive_basal)
        #al_ba.append((row["alv_prop"] * row["cor_av_liv"])/100)
        
        dead_basal = (row["ded_prop"] * row["cor_av_ded"])/100
        print("dead basal: ", dead_basal)
        ded_ba.append(dead_basal)
        #ded_ba.append((row["ded_prop"] * row["cor_av_ded"])/100)
        # Calculate site carbon
        
        c_lev = total_basal * row["leaves"]
        print("carbon leaves: ", c_lev)
        c_leaves.append(c_lev)
        c_twg = total_basal * row["twigs"]
        print("carbon twigs: ", c_twg)
        c_twigs.append(c_twg)
        
        c_bark.append(total_basal * row["bark"])
        c_wood.append(total_basal * row["wood"])
        c_branches.append(total_basal * row["branches"])
        c_stems.append(total_basal * row["stems"])
        c_agb.append(total_basal * row["agb"])
        c_roots.append(total_basal * row["root"])
        
#         c_leaves.append(row["total_prop"] * row["leaves"])
#         c_twigs.append(row["total_prop"] * row["twigs"])
#         c_bark.append(row["total_prop"] * row["bark"])
#         c_wood.append(row["total_prop"] * row["wood"])
#         c_branches.append(row["total_prop"] * row["branches"])
#         c_stems.append(row["total_prop"] * row["stems"])
#         c_agb.append(row["total_prop"] * row["agb"])
#         c_roots.append(row["total_prop"] * row["root"])

    # Total basal area    
    df["total_alv_ba"] = al_ba
    df["total_ded_ba"] = ded_ba
    df["total_ba"] = tot_ba
    # Carbon stock
    df["c_leaves"] = c_leaves
    df["c_twigs"] = c_twigs
    df["c_bark"] = c_bark
    df["c_wood"] = c_wood
    df["c_branches"] = c_branches
    df["c_stems"] = c_stems
    df["c_agb"] = c_agb
    df["c_roots"] = c_roots
    
            
    return df
        

In [86]:
df4.columns

Index(['species', 'alive', 'dead', 'alv_prop', 'ded_prop', 'total_prop',
       'site', 'uid', 'date', 'factor', 'loc_count', 'geometry', 'lon_gda94',
       'lat_gda94', 'count', 'avg_alive', 'avg_dead', 'avg_total',
       'cor_av_liv', 'cor_av_ded', 'cor_av_tot', 'co_ef_nme', 'leaves',
       'leaves_r2', 'twigs', 'bark', 'bark_r2', 'wood', 'wood_r2', 'branches',
       'branches_r2', 'stems', 'stems_r2', 'agb', 'agb_r2', 'root', 'root_r2',
       'tot_bm'],
      dtype='object')

In [87]:
df5 =total_ba_m2(df4)

row:  species                                           Buchorvet
alive                                                  11.0
dead                                                    0.0
alv_prop                                           4.545455
ded_prop                                                0.0
total_prop                                         4.365079
site                                                  lit01
uid                                                     1.0
date                                               20130426
factor                                                  0.5
loc_count                                               7.0
geometry       POINT (131.0619172006217 -12.54791228210612)
lon_gda94                                        131.061917
lat_gda94                                        -12.547912
count                                                   7.0
avg_alive                                         34.571429
avg_dead                          

row:  species                                 Eucalyptus pruinosa
alive                                                   5.0
dead                                                    5.0
alv_prop                                          71.428571
ded_prop                                          83.333333
total_prop                                        76.923077
site                                                   wh02
uid                                                    50.0
date                                               20120724
factor                                                  0.1
loc_count                                               7.0
geometry       POINT (130.6792955259559 -18.23425249677285)
lon_gda94                                        130.679296
lat_gda94                                        -18.234252
count                                                   7.0
avg_alive                                               1.0
avg_dead                          

In [88]:

df5

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,total_ded_ba,total_ba,c_leaves,c_twigs,c_bark,c_wood,c_branches,c_stems,c_agb,c_roots
0,Buchorvet,11.0,0.0,4.545455,0.0,4.365079,lit01,1.0,20130426,0.5,...,0.000000,0.785714,73.071429,76.214286,297.785714,968.785714,734.642857,1266.571429,2074.285714,425.857143
1,C. armstrongii,3.0,0.0,1.239669,0.0,1.190476,lit01,1.0,20130426,0.5,...,0.000000,0.214286,19.928571,20.785714,81.214286,264.214286,200.357143,345.428571,565.714286,116.142857
2,C. fraseri,2.0,0.0,0.826446,0.0,0.793651,lit01,1.0,20130426,0.5,...,0.000000,0.142857,13.285714,13.857143,54.142857,176.142857,133.571429,230.285714,377.142857,77.428571
3,Corymbia sp.,1.0,0.0,0.413223,0.0,0.396825,lit01,1.0,20130426,0.5,...,0.000000,0.071429,6.642857,6.928571,27.071429,88.071429,66.785714,115.142857,188.571429,38.714286
4,E. chlorostachys,38.0,2.0,15.702479,20.0,15.873016,lit01,1.0,20130426,0.5,...,0.142857,2.857143,440.000000,457.142857,1145.714286,2982.857143,2325.714286,4128.571429,6894.285714,1548.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,No basal,0.0,0.0,0.000000,0.0,0.000000,lim08,17.0,20130601,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
44,No basal,0.0,0.0,0.000000,0.0,0.000000,lim09,18.0,20130601,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45,No basal,0.0,0.0,0.000000,0.0,0.000000,ant09,39.0,20120626,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
46,No basal,0.0,0.0,0.000000,0.0,0.000000,ant07,38.0,20120626,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [89]:
df5.columns

Index(['species', 'alive', 'dead', 'alv_prop', 'ded_prop', 'total_prop',
       'site', 'uid', 'date', 'factor', 'loc_count', 'geometry', 'lon_gda94',
       'lat_gda94', 'count', 'avg_alive', 'avg_dead', 'avg_total',
       'cor_av_liv', 'cor_av_ded', 'cor_av_tot', 'co_ef_nme', 'leaves',
       'leaves_r2', 'twigs', 'bark', 'bark_r2', 'wood', 'wood_r2', 'branches',
       'branches_r2', 'stems', 'stems_r2', 'agb', 'agb_r2', 'root', 'root_r2',
       'tot_bm', 'total_alv_ba', 'total_ded_ba', 'total_ba', 'c_leaves',
       'c_twigs', 'c_bark', 'c_wood', 'c_branches', 'c_stems', 'c_agb',
       'c_roots'],
      dtype='object')

In [90]:
df5.to_csv(os.path.join(output_dir, "df5_v6.csv"), index=False)

In [91]:
df5.sample(2)

,species,alive,dead,alv_prop,ded_prop,total_prop,site,uid,date,factor,...,total_ded_ba,total_ba,c_leaves,c_twigs,c_bark,c_wood,c_branches,c_stems,c_agb,c_roots
48,Acacia valida,19.0,0.0,26.027397,NaN,26.027397,auv03,119.0,20121010,0.1,...,NaN,0.271429,25.242857,26.328571,102.871429,334.671429,253.785714,437.542857,716.571429,147.114286
12,Syzygium suborbiculare,8.0,0.0,3.305785,0.0,3.174603,lit01,1.0,20130426,0.5,...,0.0,0.571429,53.142857,55.428571,216.571429,704.571429,534.285714,921.142857,1508.571429,309.714286


In [104]:
def total_kg_per_site(df):
    df_list = []

    for uid in df.uid.unique():
        df1 = df[df["uid"]==uid]
        # -------------------------------- Biomass -----------------------------------
        df1["bio_l_kg1ha"] = (df1.loc[:, "c_leaves"].sum())*0.47
        
        df1["bio_t_kg1ha"] = (df1.loc[:, "c_twigs"].sum())*0.49
        
        df1["bio_b_kg1ha"] = (df1.loc[:, "c_bark"].sum())*0.49
        
        df1["bio_w_kg1ha"] = (df1.loc[:, "c_wood"].sum())*0.49
        
        df1["bio_br_kg1ha"] = (df1.loc[:, "c_branches"].sum())*0.49
        
        df1["bio_s_kg1ha"] = (df1.loc[:, "c_stems"].sum())*0.49
        
        df1["bio_r_kg1ha"] = (df1.loc[:, "c_roots"].sum())*0.49
        
        df1["bio_agb_kg1ha"] = (df1.loc[:, "c_agb"].sum())*0.49
        
        # ------------------------------ Carbon ------------------------------------
        
        df1["c_l_kg1ha"] = (df1.loc[:, "c_leaves"].sum())
        
        
        df1["c_t_kg1ha"] = (df1.loc[:, "c_twigs"].sum())
        
        df1["c_b_kg1ha"] = (df1.loc[:, "c_bark"].sum())
        
        df1["c_w_kg1ha"] = (df1.loc[:, "c_wood"].sum())
        
        df1["c_br_kg1ha"] = (df1.loc[:, "c_branches"].sum())
        
        df1["c_s_kg1ha"] = (df1.loc[:, "c_stems"].sum())

        df1["c_r_kg1ha"] = (df1.loc[:, "c_roots"].sum())
        
        df1["c_agb_kg1ha"] = (df1.loc[:, "c_agb"].sum())
        
        print(df1)
        df_list.append(df1)

    
    final_df = pd.concat(df_list)        
    return final_df

In [105]:
df5 = total_kg_per_site(df5)

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

                   species  alive  dead   alv_prop  ded_prop  total_prop  \
0                Buchorvet   11.0   0.0   4.545455       0.0    4.365079   
1           C. armstrongii    3.0   0.0   1.239669       0.0    1.190476   
2               C. fraseri    2.0   0.0   0.826446       0.0    0.793651   
3             Corymbia sp.    1.0   0.0   0.413223       0.0    0.396825   
4         E. chlorostachys   38.0   2.0  15.702479      20.0   15.873016   
5               E. miniata   18.0   0.0   7.438017       0.0    7.142857   
6            E. tetrodonta   98.0   4.0  40.495868      40.0   40.476190   
7              F. achleata    3.0   0.0   1.239669       0.0    1.190476   
8               L. humilis    5.0   0.0   2.066116       0.0    1.984127   
9                P. careya    6.0   0.0   2.479339       0.0    2.380952   
10            P. pubescens    1.0   0.0   0.413223       0.0    0.396825   
11             P. spiralis   48.0   4.0  19.834711      40.0   20.634921   
12  Syzygium

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

                      species  alive  dead   alv_prop  ded_prop  total_prop  \
189        Buchanania obovata    1.0   0.0   0.645161       0.0    0.617284   
190            C. armstrongii    4.0   0.0   2.580645       0.0    2.469136   
191       Corymbia polysciada    1.0   0.0   0.645161       0.0    0.617284   
192         Corymbia porrecta    7.0   0.0   4.516129       0.0    4.320988   
193          E. chlorostachys   18.0   0.0  11.612903       0.0   11.111111   
194                E. miniata   57.0   0.0  36.774194       0.0   35.185185   
195             E. tetrodonta   38.0   0.0  24.516129       0.0   23.456790   
196       Gardenia megasperma    1.0   0.0   0.645161       0.0    0.617284   
197    Lophostemon lactifluus    3.0   0.0   1.935484       0.0    1.851852   
198                 P. careya    1.0   0.0   0.645161       0.0    0.617284   
199  Terminalia ferdinandiana   24.0   0.0  15.483871       0.0   14.814815   
200                      dead    0.0   7.0   0.00000

    species  alive  dead  alv_prop  ded_prop  total_prop   site    uid  \
8  No basal    0.0   0.0       0.0       0.0         0.0  nt004  127.0   

       date  factor  ...  bio_r_kg1ha bio_agb_kg1ha  c_l_kg1ha  c_t_kg1ha  \
8  20120524     0.0  ...          0.0           0.0        0.0        0.0   

   c_b_kg1ha  c_w_kg1ha  c_br_kg1ha  c_s_kg1ha  c_r_kg1ha  c_agb_kg1ha  
8        0.0        0.0         0.0        0.0        0.0          0.0  

[1 rows x 65 columns]
    species  alive  dead  alv_prop  ded_prop  total_prop   site   uid  \
9  No basal    0.0   0.0       0.0       0.0         0.0  amg03  26.0   

       date  factor  ...  bio_r_kg1ha bio_agb_kg1ha  c_l_kg1ha  c_t_kg1ha  \
9  20120620     0.0  ...          0.0           0.0        0.0        0.0   

   c_b_kg1ha  c_w_kg1ha  c_br_kg1ha  c_s_kg1ha  c_r_kg1ha  c_agb_kg1ha  
9        0.0        0.0         0.0        0.0        0.0          0.0  

[1 rows x 65 columns]
     species  alive  dead  alv_prop  ded_prop  total_pro

C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\rmcgr\Miniconda3\envs\biomass_zonal\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [94]:
df5.shape

(353, 65)

In [95]:
df5.to_csv(os.path.join(output_dir, "slats_kg_1ha_bio_c_v6.csv"), index=False)

In [96]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 0 to 47
Data columns (total 65 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   species        353 non-null    object 
 1   alive          353 non-null    float64
 2   dead           353 non-null    float64
 3   alv_prop       353 non-null    float64
 4   ded_prop       331 non-null    float64
 5   total_prop     353 non-null    float64
 6   site           353 non-null    object 
 7   uid            353 non-null    float64
 8   date           353 non-null    object 
 9   factor         353 non-null    float64
 10  loc_count      353 non-null    float64
 11  geometry       353 non-null    object 
 12  lon_gda94      353 non-null    float64
 13  lat_gda94      353 non-null    float64
 14  count          353 non-null    float64
 15  avg_alive      353 non-null    float64
 16  avg_dead       353 non-null    float64
 17  avg_total      353 non-null    float64
 18  cor_av_liv 

In [97]:
df_totals = df5[["uid", "site", "date", "lon_gda94", "lat_gda94", "bio_l_kg1ha", "bio_t_kg1ha", "bio_b_kg1ha", "bio_w_kg1ha",
                 "bio_br_kg1ha", "bio_s_kg1ha", "bio_r_kg1ha", "bio_agb_kg1ha", "c_l_kg1ha", "c_t_kg1ha",
                 "c_b_kg1ha", "c_w_kg1ha", "c_br_kg1ha", "c_s_kg1ha", "c_r_kg1ha", "c_agb_kg1ha"]]

In [98]:
# df_final[df_final["uid"] == 1]

In [99]:
df_totals.drop_duplicates(subset=["site"], inplace=True)
df_totals.to_csv(os.path.join(output_dir, "slats_c_bio_site_totals_v6.csv"), index=False)

C:\Users\rmcgr\AppData\Roaming\Python\Python37\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
